In [1]:
import gradio as gr
import pandas as pd
import mlflow
import numpy as np
import mlflow.pyfunc
import xgboost as xgb
import mlflow.xgboost
import pickle
from gradio_functions import *
import matplotlib.pyplot as plt
import plotly.express as px
from model_functions import *
import html
mlflow.set_tracking_uri("http://localhost:5000")

c:\Python311\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Python311\Lib\site-packages\visions\backends\shared\nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


In [2]:
with gr.Blocks() as demo:
     with gr.Tab(label="Admin", default=True):
        gr.Markdown(
            """
        # Admin-Page
        """
        )
        gr.Markdown("## Admin page")
        link = "https://michaelseitz98.github.io/enterprise-ai-project/eda-wue-rent-all.html"
        description = (
            "Explorative Data Analyis for Flats to rent"
        )
        html_code = f'<a href="{html.escape(link)}" font-size: 16px;">{html.escape(description)}</a>'
        gr.HTML(html_code)
        gr.HTML("<hr>")
        gr.Markdown("## Retrain")
        gr.Markdown("Select the models you want to retrain.")

        with gr.Row():
            xgb = gr.Checkbox(label="xgb")
            rf = gr.Checkbox(label="rf")
            linear = gr.Checkbox(label="linear", value=True)
            lasso = gr.Checkbox(label="lasso")
            ridge = gr.Checkbox(label="ridge")
            elasticnet = gr.Checkbox(label="elasticnet")
            baseline = gr.Checkbox(label="baseline-rent", value=True)

        model_list = []
        if xgb:
            model_list.append("xgb")
        if rf:
            model_list.append("rf")
        if lasso:
            model_list.append("lasso")
        if ridge:
            model_list.append("ridge")
        if elasticnet:
            model_list.append("elasticnet")
        if baseline:
            model_list.append("baseline-rent")

        limit = gr.Slider(label="Amount of scraped slides", value=1, maximum=10)
        nachtraining = gr.Button("Scrape new data, retrain and evaluate models")
        
        df_results = gr.Dataframe(label="Results of retraining")
        output_plot = gr.Plot(visible=False)

        nachtraining.click(fn=gradio_retrain_with_added_data, inputs=[xgb, ridge, rf, elasticnet, linear, lasso, baseline, limit], outputs=[df_results, output_plot])


    with gr.Tab(label="Immopreisrechner"):
        gr.Markdown(
            """
        # Geben Sie hier an, welche Merkmale Ihre Immobilie besitzt 🏠
        """
        )
        with gr.Row():
            feature_zip = gr.Dropdown(
                label="Postleitzahl",
                choices=[
                    "97070 Würzburg-Altstadt",
                    "97072 Würzburg-Sanderau",
                    "97074 Würzburg-Frauenland",
                    "97076 Würzburg-Lengfeld",
                    "97078 Würzburg-Lindleinsmühle",
                    "97080 Würzburg-Dürrbachtal",
                    "97082 Würzburg-Steinbachtal",
                    "97084 Würzburg-Heuchelhof",
                    "97204 Höchberg",
                    "97209 Veitshöchheim",
                    "97218 Gerbrunn",
                    "97222 Rimpar",
                    "97228 Rottendorf",
                    "97234 Reichenberg",
                    "97236 Randersacker",
                    "97246 Eibelstadt",
                    "97249 Eisingen",
                    "97250 Erlabrunn",
                    "97261 Güntersleben",
                    "97270 Kist",
                    "97288 Theilheim",
                    "97297 Waldbüttelbrunn",
                    "97299 Zell am Main",
                ],
                value="97070 Würzburg-Altstadt",
            )
        with gr.Row():
            feature_squrmeter = gr.Number(label="Wohnfläche [qm]", value=79)
        with gr.Row():
            feature_rooms = gr.Number(label="Anzahl der Zimmer", value=3)
        with gr.Row():
            features_altbau = gr.Checkbox(label="Altbau (bis 1945)")
            feature_balkon = gr.Checkbox(label="Balkon")
            feature_ba = gr.Checkbox(label="Barrierefrei")
            feature_dachgeschoss = gr.Checkbox(label="Dachgeschoss")
            feature_einbaukueche = gr.Checkbox(label="Einbauküche")
            feature_neubau = gr.Checkbox(label="Neubau")
            feature_parkett = gr.Checkbox(label="Parkett")
            feature_stellplatz = gr.Checkbox(label="Stellplatz")
            feature_badwc_getrennt = gr.Checkbox(label="Bad/WC getrennt")
            feature_personenaufzug = gr.Checkbox(label="Personenaufzug")
            feature_garten = gr.Checkbox(label="Garten")
            feature_garage = gr.Checkbox(label="Garage")
            feature_renoviert = gr.Checkbox(label="Renoviert")
            feature_terrasse = gr.Checkbox(label="Terrasse")
            feature_wanne = gr.Checkbox(label="Badewanne")
            feature_zentralheizung = gr.Checkbox(label="Zentralheizung")
            feature_abstellraum = gr.Checkbox(label="Abstellraum")
            feature_fernwaerme = gr.Checkbox(label="Fernwärme")
            feature_fussbodenheizung = gr.Checkbox(label="Fußbodenheizung")
            feature_gartenmitbenutzung = gr.Checkbox(label="Gartenmitbenutzung")
            feature_kellerabteil = gr.Checkbox(label="Kellerabteil")

        with gr.Accordion("erweiterte Optionen", open=False):
            erklärung = gr.Checkbox(label="XGBoost vorhersagen")

        btn = gr.Button("Berechne deinen Preis")
        output_html = gr.HTML(label="Dein Preis")
        btn.click(
            fn=trigger_actions,
            inputs=[
                feature_squrmeter,
                feature_zip,
                feature_rooms,
                features_altbau,
                feature_balkon,
                feature_ba,
                feature_dachgeschoss,
                feature_einbaukueche,
                feature_neubau,
                feature_parkett,
                feature_stellplatz,
                feature_badwc_getrennt,
                feature_personenaufzug,
                feature_garten,
                feature_garage,
                feature_renoviert,
                feature_terrasse,
                feature_wanne,
                feature_zentralheizung,
                feature_abstellraum,
                feature_fernwaerme,
                feature_fussbodenheizung,
                feature_gartenmitbenutzung,
                feature_kellerabteil,
                erklärung,
            ],
            outputs=[output_html],
        )

   
demo.queue(concurrency_count=10).launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


https://www.immowelt.de/liste/wuerzburg/wohnungen/mieten
started
Retraining data successfully scraped.
Retraining data successfully written to excel under data/retrain_train_data.xslx
Done with raw preprocessing.
old shape of train_recent (108, 47)
Index(['Object_price', 'LivingSpace', 'Rooms', 'ConstructionYear', 'ZipCode',
       'EstateType', 'DistributionType', 'abstellraum', 'altbau_(bis_1945)',
       'balkon', 'barriefrei', 'dachgeschoss', 'dsl', 'dusche', 'einbaukueche',
       'elektro', 'erdgeschoss', 'erdwaerme', 'etagenheizung', 'fenster',
       'fern', 'ferne', 'fliesen', 'frei', 'fussbodenheizung', 'gaestewc',
       'garage', 'gas', 'gepflegt', 'granit', 'haustiere_erlaubt',
       'kable_sat_tv', 'kelleranteil',
       'kontrollierte_be-_und_entlueftungsanlage', 'kunststofffenster',
       'laminat', 'loggia', 'luftwp', 'moebliert', 'neubau', 'ofenheizung',
       'pantry', 'parkett', 'pellet', 'personenaufzug', 'reinigung',
       'renoviert', 'souterrain', 'stein', '

2023/07/16 14:47:35 INFO mlflow.tracking.fluent: Experiment with name 'retraining_2023-07-16_14-47' does not exist. Creating a new experiment.


XGB------
train(122, 46)
val:(27, 46)
y_train:(122,)
y_val:(27, 1)
[0]	validation_0-rmse:949.12014	validation_0-mae:827.12919
[1]	validation_0-rmse:708.56292	validation_0-mae:592.37173
[2]	validation_0-rmse:542.39578	validation_0-mae:424.33665
[3]	validation_0-rmse:436.03012	validation_0-mae:315.96783
[4]	validation_0-rmse:370.09318	validation_0-mae:267.51430
[5]	validation_0-rmse:332.66945	validation_0-mae:251.24185
[6]	validation_0-rmse:312.07361	validation_0-mae:237.65018
[7]	validation_0-rmse:308.48192	validation_0-mae:232.77575
[8]	validation_0-rmse:303.98871	validation_0-mae:229.11343
[9]	validation_0-rmse:299.81530	validation_0-mae:223.49278
[10]	validation_0-rmse:296.92310	validation_0-mae:220.62628
[11]	validation_0-rmse:297.91061	validation_0-mae:217.56628
[12]	validation_0-rmse:298.63138	validation_0-mae:218.46386
[13]	validation_0-rmse:300.94673	validation_0-mae:217.15734
[14]	validation_0-rmse:300.60296	validation_0-mae:217.42990
[15]	validation_0-rmse:302.99825	validation

2023/07/16 14:47:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python311\Lib\site-packages\mlflow\models\signature.py:144: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/16 14:48:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python311\Lib\site-packages\_distutils_hack\__init__.py:33:

...

Training lasso model done...
---EVALUATION AND LOGGING TO MLFLOW------ lasso


2023/07/16 14:52:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python311\Lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/16 14:52:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Python311\Lib\site-packages\mlflow\data\pandas_dataset.p

BASELINE-RENT------
Extrcated rental price per square meter via scraper: 11.21
Average rental price per sqm: 11.21
Baseline Mae: 285.5564
Baseline Mae: 285.5564
Baseline MSE: 140135.41872988813
Baseline R2 Score: 0.5802888162379594
Training baseline-rent model done...
---EVALUATION AND LOGGING TO MLFLOW------ baseline-rent


The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Done with retraining:             model     mae        mse    r2  mae_test  mse_test  r2_test  \
0            xgb  216.29   91807.94  0.73    141.55  42542.08     0.86   
1          ridge  240.77   92518.07  0.72    179.84  60708.98     0.79   
2             rf  204.19   88573.62  0.73    141.07  34304.19     0.88   
3     elasticnet  241.01   99799.93  0.70    187.53  51435.20     0.83   
4         linear  275.89  124804.47  0.63    186.34  76205.55     0.74   
5          lasso  244.99   95669.21  0.71    187.91  65708.14     0.78   
6  baseline-rent  285.56  140135.42  0.58    237.83  92491.55     0.69   

   mae_train  mse_train  r2_train  
0      39.59    2734.17      0.99  
1     129.29   28652.93      0.92  
2      61.46    7247.01      0.98  
3     179.20   53210.63      0.86  
4     122.39   26955.37      0.93  
5     130.03   28868.93      0.92  
6        NaN        NaN       NaN  
Save results to excel
Done with saving results to excel
Done with plotting:  Figure({
    'data'

In [ ]:
import gradio as gr
import time

def slowly_reverse(word, progress=gr.Progress()):
    progress(0, desc="Starting")
    time.sleep(1)
    progress(0.5, desc="Halfway")
    time.sleep(1)
    new_string = ""
    for letter in progress.tqdm(word, desc="Reversing"):
        time.sleep(0.25)
        new_string = letter + new_string
    return new_string

demo = gr.Interface(slowly_reverse, gr.Text(), gr.Text())

if __name__ == "__main__":
    demo.queue(concurrency_count=10).launch()